<a href="https://colab.research.google.com/github/aaghash17/price_action_test/blob/main/Price%20Action%20Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd

In [ ]:
def get_day_data(symbol, date):
  ticker = yf.Ticker(symbol)
  start_date = datetime.strptime(date, "%Y-%m-%d")
  end_date = start_date + timedelta(days=1)
  try:
    data = ticker.history(start=start_date, end=end_date, interval="1m");
  except Exception as e:
    return None
  return data

In [ ]:
def get_range_data(ticker,start_date, end_date):
  start_date = datetime.strptime(start_date, "%Y-%m-%d")
  end_date = datetime.strptime(end_date, "%Y-%m-%d")
  data_list = []
  current_date = start_date
  while current_date <= end_date:
    try:
      data = get_day_data(ticker, current_date.strftime("%Y-%m-%d"))
    except Exception as e:
      return pd.DataFrame()
    if data is not None and not data.empty:
      data_list.append(data)
    current_date += timedelta(days=1)
  if data_list:
    return pd.concat(data_list)
  return pd.DataFrame()

In [ ]:
ticker_symbol = "TATAMOTORS.NS"

start_date = "2025-08-08"
end_date = "2025-08-08"

historical_data = get_range_data(ticker_symbol, start_date, end_date)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=historical_data.index,
                open=historical_data['Open'],
                high=historical_data['High'],
                low=historical_data['Low'],
                close=historical_data['Close'])])

fig.update_layout(
    title=f'{ticker_symbol} Candlestick Chart',
    xaxis_title='Date',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    xaxis=dict(
        type='date',
        tickformat='%H:%M'  # Format to show hours and minutes
    )
)

fig.show()

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

def find_collinear_points_dataframe(data, tolerance=0.1):
    if len(data) < 3:
        return pd.DataFrame()

    all_points = historical_data[[index, 'High']].values

    test = combinations(points, )

    for p1,p2,p3 in test:
      print(p1,p2,p3)

    for p1, p2, p3 in combinations(points, 3):
      x1, y1 = p1
      x2, y2 = p2
      x3, y3 = p3

      # Handle vertical lines separately to avoid division by zero
      if (x2 - x1) == 0:
        if abs(x3 - x2) < tolerance:
          # Found a vertical line, now find all points on this line
          collinear_points = data[np.isclose(data[x_col], x1, atol=tolerance)]
          if len(collinear_points) >= 3:
            print(f"Found {len(collinear_points)} collinear points on a vertical line.")
            return collinear_points

      else:
        slope1 = (y2 - y1) / (x2 - x1)
        if (x3 - x2) != 0:
          slope2 = (y3 - y2) / (x3 - x2)

        if abs(slope1 - slope2) < tolerance:
          intercept = y1 - slope1 * x1

          is_on_line = np.isclose(data[y_col], slope1 * data[x_col] + intercept, atol=tolerance)
          collinear_points = data[is_on_line]

          if len(collinear_points) >= 3:
            print(f"Found {len(collinear_points)} collinear points on a line with slope {slope1:.2f}.")
            return collinear_points

    print("No set of three or more collinear points found.")
    return pd.DataFrame()

# 2. Call the function to get the DataFrame of collinear points.
collinear_df = find_collinear_points_dataframe(historical_data)



KeyError: "['index'] not in index"

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=historical_data.index,
    y=historical_data['High'],
    mode='markers',
    name='High Prices',
    marker=dict(color='green', size=4)
))

# Add a scatter plot for the Low values using markers
fig.add_trace(go.Scatter(
    x=historical_data.index,
    y=historical_data['Low'],
    mode='markers',
    name='Low Prices',
    marker=dict(color='red', size=4)
))

# Customize the chart layout
fig.update_layout(
    title='High and Low Prices (Markers Only)',
    xaxis_title='Date',
    yaxis_title='Price',
)

fig.show()